In [1]:
import json

reports = []
with open('../data/ufodata.json') as f:
    for i, k in enumerate(f):
#         if (i % 100 == 0):
#             print(i)
        try:
            reports.append(json.loads(k))
        except:
            continue

In [2]:
reports[0]

{'_id': {'$oid': '591382287c40e83fd74240bd'},
 'url': 'http://www.nuforc.org/webreports/133/S133978.html',
 'html': '<html xmlns:v="urn:schemas-microsoft-com:vml"\r\nxmlns:o="urn:schemas-microsoft-com:office:office"\r\nxmlns:w="urn:schemas-microsoft-com:office:word"\r\nxmlns:m="http://schemas.microsoft.com/office/2004/12/omml"\r\nxmlns="http://www.w3.org/TR/REC-html40">\r\n\r\n<head>\r\n<meta http-equiv=Content-Type content="text/html; charset=windows-1252">\r\n<meta name=ProgId content=Word.Document>\r\n<meta name=Generator content="Microsoft Word 15">\r\n<meta name=Originator content="Microsoft Word 15">\r\n<link rel=File-List href="STemplate_files/filelist.xml">\r\n<!--[if gte mso 9]><xml>\r\n <o:DocumentProperties>\r\n  <o:Author>Christian Stepien</o:Author>\r\n  <o:LastAuthor>Christian Stepien</o:LastAuthor>\r\n  <o:Revision>3</o:Revision>\r\n  <o:TotalTime>1</o:TotalTime>\r\n  <o:Created>2014-02-14T21:30:00Z</o:Created>\r\n  <o:LastSaved>2014-02-14T21:35:00Z</o:LastSaved>\r\n  <o

In [3]:
type(reports)

list

In [4]:
# finds each key in json file for that record
print(reports[0].keys(), '\n')

# prints each key and value (using f strings)
for k, v in reports[0].items():
    print(f'{k}: {v} \n')

dict_keys(['_id', 'url', 'html', 'time']) 

_id: {'$oid': '591382287c40e83fd74240bd'} 

url: http://www.nuforc.org/webreports/133/S133978.html 

html: <html xmlns:v="urn:schemas-microsoft-com:vml"
xmlns:o="urn:schemas-microsoft-com:office:office"
xmlns:w="urn:schemas-microsoft-com:office:word"
xmlns:m="http://schemas.microsoft.com/office/2004/12/omml"
xmlns="http://www.w3.org/TR/REC-html40">

<head>
<meta http-equiv=Content-Type content="text/html; charset=windows-1252">
<meta name=ProgId content=Word.Document>
<meta name=Generator content="Microsoft Word 15">
<meta name=Originator content="Microsoft Word 15">
<link rel=File-List href="STemplate_files/filelist.xml">
<!--[if gte mso 9]><xml>
 <o:DocumentProperties>
  <o:Author>Christian Stepien</o:Author>
  <o:LastAuthor>Christian Stepien</o:LastAuthor>
  <o:Revision>3</o:Revision>
  <o:TotalTime>1</o:TotalTime>
  <o:Created>2014-02-14T21:30:00Z</o:Created>
  <o:LastSaved>2014-02-14T21:35:00Z</o:LastSaved>
  <o:Pages>1</o:Pages>
  <o:Wo

In [5]:
# div = soup.find("div", {"class": "scoreTable"})
# table = div.find("table")

# # This returns an iterator over the rows in the table.
# rows = table.find_all("tr")

# all_rows = []

# # Let's store each row as a dictionary 
# empty_row = {
#     "rank": None, "player": None, "time": None, "comment": None
# }

# # The first row contains header information, so we are skipping it.
# for row in rows[1:]:
#     new_row = copy.copy(empty_row)
#     # A list of all the entries in the row.
#     columns = row.find_all("td")
#     new_row['rank'] = int(columns[0].text.strip())
#     new_row['player'] = columns[1].text.strip()
#     new_row['time'] = columns[2].text.strip()
#     new_row['comment'] = columns[4].text.strip()
#     all_rows.append(new_row)

In [6]:
import json # to work with json file format
from bs4 import BeautifulSoup # to parse html
import re

In [7]:
len(reports)

136492

In [ ]:
soups = []
for i in range(15000):
    soup = BeautifulSoup(reports[i]['html'], 'html.parser')
    table = soup.find_all('font')
    
    document = []
    for i in range(len(table)):
        text = ''.join(str(table[i]))
        document.append(text)

    # remember to import re
    cleaned = []
    for line in document:
        cleaned.append(re.sub('<.*?>', ' ', line))

    soups.append(cleaned)

In [ ]:
for entry in soups[:2]:
    print(entry[3])

In [ ]:
len(soups)

In [ ]:
# extracting features from second row

# def extract_feats(text, row, stop):
#     string = row
#     text_str = re.findall(f'{re.escape(text)}: (.*) {re.escape(stop)}', string)
#     return text_str

def extract_feats(text, row, stop):
    string = row
    text_str = re.findall(f'(?<={re.escape(text)}:).*(?={re.escape(stop)})', string)
    return text_str

In [ ]:
text=['Occurred ', 'Location', 'Shape', 'Duration', 'Note']
stop = ['(Entered', 'Shape', 'Duration', 's ', 'PD']

col_values = []

for entry in soups:
    try:
        row = entry[2]
        observation = entry[3]
        occur = extract_feats(text[0], row, stop[0])[0]
        location = extract_feats(text[1], row, stop[1])[0]
        shape = extract_feats(text[2], row, stop[2])[0]
        duration = extract_feats(text[3], row, stop[3])[0]
        if 'Note' not in observation:
            note = []
        else:
            note = extract_feats(text[4], observation, stop[-1])[0]
        col_values.append([occur, location, shape, duration, observation, note])
    except:
        continue

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

plt.style.use('ggplot')

In [15]:
data = pd.read_csv('data/ufodata.csv', index_col=0)

In [16]:
print('Shape of UFO dataframe: ', data.shape)
data.head(20)

Shape of UFO dataframe:  (11931, 6)


0                               1            2  \
0    5/6/2017 05:00                Camp McGregor, NM        Light    
1    5/5/2017 11:30                       Austin, TX         Disk    
2    5/5/2017 03:00                    El Mirage, AZ       Circle    
3    5/4/2017 21:27                      Phoenix, AZ       Circle    
4    5/4/2017 18:30                      Phoenix, AZ     Teardrop    
5    5/4/2017 04:50                         Taft, CA     Changing    
6    5/4/2017 00:00                  Goose Creek, SC       Circle    
7         5/4/2017                      Savannah, GA       Sphere    
8    5/3/2017 23:15                   West Union, OH       Circle    
9    5/3/2017 21:00                     Portland, OR       Circle    
10   5/3/2017 15:00                     Loveland, CO     Triangle    
11   5/3/2017 11:15                    Barneveld, WI     Triangle    
12   5/3/2017 08:50                      Phoenix, AZ        Other    
13   5/3/2017 08:00     New York City (Brooklyn), NY    Rectangle    
14   5/2/2017 22:15                  Albuquerque, NM       Sphere    
15   5/2/2017 21:50                  Albuquerque, NM       Sphere    
16   5/2/2017 21:00                     Kennesaw, GA        Light    
17   5/2/2017 17:10                  Albuquerque, NM         Oval    
18   5/2/2017 13:15                  Canoga Park, CA       Sphere    
19   5/2/2017 13:00            Edmonton (Canada), AB        Light    

                      3                                                  4  \
0             10 minute   Light seen over mountain's east of Camp McGre...   
1              3 second   Flying saucer descends, possibly lands in Nor...   
2             30 second   While letting my dog out, a very bright white...   
3             15 second   Orange round sphere.  Orange glowing sphere f...   
4              5 minute   Flying corkscrews  Looking to th east at abou...   
5             20 second   I'm a truck driver and I've seen the reddish/...   
6   3 minutes 53 second   For a little over a month, I would go outside...   
7              3 minute   Watched a 3-4 in sphere come in one end of de...   
8              5 minute   I was walking my dog and I looked up at the s...   
9             20 minute   Three moving bright lights.  There were three...   
10             4 minute   I seen two objects dancing high in the sky......   
11          5-10 minute   Very bright triangular shaped object.  I was ...   
12             2 second   MAN SEEN AIRLINE LIKE OBJECT VANISH INSIDE CO...   
13            10 minute   UFO.  A very large size, gray color, square s...   
14             4 second   Albuquerque –Around 11:15 pm.  We saw a singl...   
15             5 second   This object was flying over the mountains and...   
16               4 hour   Strange craft appears at dusk, lights up and ...   
17            10 second   Stepped out my backdoor and saw a reflective ...   
18             5 minute   Above the SF Valley  Black object way up in t...   
19             1 minute   where are all the pictures....with everyone o...   

                                                    5  
0     Report appears to us to be consistent with t...  
1     We would like to communicate with this witne...  
2                                                  []  
3     We have amended the time above, to reflect a...  
4     Source of the report elects to remain anonym...  
5     Witness indicates "Taft, Indiana" in origina...  
6     Source of the report elects to remain anonym...  
7     For some reason, the source of the report do...  
8                                                  []  
9                                                  []  
10    Source of the report elects to remain anonym...  
11    Source of the report elects to remain anonym...  
12                                                 []  
13                                                 []  
14                                                 []  
15            

In [17]:
data.columns = ['datetime', 'location', 'shape', 'duration', 'report', 'note']

In [18]:
# ufo.to_csv('data/ufodata.csv')

In [19]:
data.head()

datetime             location       shape   duration  \
0   5/6/2017 05:00     Camp McGregor, NM       Light   10 minute   
1   5/5/2017 11:30            Austin, TX        Disk    3 second   
2   5/5/2017 03:00         El Mirage, AZ      Circle   30 second   
3   5/4/2017 21:27           Phoenix, AZ      Circle   15 second   
4   5/4/2017 18:30           Phoenix, AZ    Teardrop    5 minute   

                                              report  \
0   Light seen over mountain's east of Camp McGre...   
1   Flying saucer descends, possibly lands in Nor...   
2   While letting my dog out, a very bright white...   
3   Orange round sphere.  Orange glowing sphere f...   
4   Flying corkscrews  Looking to th east at abou...   

                                                note  
0    Report appears to us to be consistent with t...  
1    We would like to communicate with this witne...  
2                                                 []  
3    We have amended the time above, to reflect a...  
4    Source of the report elects to remain anonym...